# Ast Nodes Types


for each statement node in the spannerlog grammar, this module contains a matching class that can represent
that statement in the abstract syntax tree. classes representations for relations are also included.

these classes are useful as they represent a statement with a single instance, instead of a lark tree,
thus simplifying the code required for semantic checks and manipulations of the statement.

In [ ]:
#| default_exp ast_node_types

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#| export
from nbdev.showdoc import show_doc

In [ ]:
#| export
#| output: false
from typing import List, Tuple, Set, Union, Sequence
from spannerlib.primitive_types import DataTypes, DataTypeMapping, Span

Installation NLP failed


In [ ]:
#| export
def get_term_list_string(term_list: Sequence[DataTypeMapping.term], #the term list to be turned into a string
                         type_list: Sequence[DataTypes] #the types of the terms in term_list
                         ) -> str: #a string representation of the term list
    """
    returns a string representation of the term list.
    quotes are added to string terms so they will not be confused with variables. <br>
    @raise Exception: if length of term list doesn't match the length of type list.
    """
    if len(term_list) != len(type_list):
        raise Exception(f"received different lengths of term_list ({len(term_list)}) "
                        f"and type_list ({len(type_list)})")
        
    terms_with_quoted_strings = [f'"{term}"' if term_type is DataTypes.string
                                 else str(term)
                                 for term, term_type in zip(term_list, type_list)]
    term_list_string = ', '.join(terms_with_quoted_strings)
    return term_list_string

::: {.callout-note collapse="true"}

##### Example

In [ ]:
term_list = ["X", "jack", 1]
type_list = [DataTypes.free_var_name, DataTypes.string, DataTypes.integer]
print(get_term_list_string(term_list,type_list))

X, "jack", 1


:::

In [ ]:
#| export
class RelationDeclaration:
    """a representation of a relation_declaration statement"""

    def __init__(self, relation_name: str, # the name of the relation.
                 type_list: Sequence[DataTypes]): # a list of the types of the terms in the relation's tuples.
        """
        @raise Exception: if there is invalid term type in term list.
        """
        self.relation_name = relation_name
        self.type_list = type_list
        
    def __str__(self) -> str:
        type_strings = []
        for term_type in self.type_list:
            if term_type is DataTypes.string:
                type_strings.append('str')
            elif term_type is DataTypes.span:
                type_strings.append('span')
            elif term_type is DataTypes.integer:
                type_strings.append('int')
            elif term_type is DataTypes.free_var_name:
                type_strings.append('free_var_name')
            else:
                raise ValueError(f"invalid term type ({term_type})")

        type_list_string = ', '.join(type_strings)
        relation_declaration_string = f"{self.relation_name}({type_list_string})"
        return relation_declaration_string

    def __repr__(self) -> str:
        return str(self)

::: {.callout-note collapse="true"}

##### Example

In [ ]:
rel_decl = RelationDeclaration("parent", [DataTypes.string, DataTypes.string])
print(rel_decl)

parent(str, str)


:::


In [ ]:
#| export
class Relation:
    """a representation of a normal relation"""

    def __init__(self, relation_name: str, # the name of the relation
                 term_list: Sequence[DataTypeMapping.term], # a list of the relation terms
                 type_list: Sequence[DataTypes]): # a list of the relation term types
        """
        @raise Exception: if length of term list doesn't match the length of type list.
        """
        if len(term_list) != len(type_list):
            raise Exception(f"received different lengths of term_list ({len(term_list)}) "
                            f"and type_list ({len(type_list)})")
        self.relation_name = relation_name
        self.term_list = term_list
        self.type_list = type_list
        
    def __str__(self) -> str:
        term_list_string = get_term_list_string(self.term_list, self.type_list)
        relation_string = f"{self.relation_name}({term_list_string})"
        return relation_string

    def __repr__(self) -> str:
        return str(self)

    def get_term_list(self) -> Sequence[DataTypeMapping.term]:
        return self.term_list

    def get_type_list(self) -> Sequence[DataTypes]:
        return self.type_list

    def get_select_cols_values_and_types(self
            )-> set: # a set of constant variables that should be selected from the relation
        col_value_type = set()
        for i, (var_type, value) in enumerate(zip(self.type_list, self.term_list)):
            if var_type != DataTypes.free_var_name:
                col_value_type.add((i, value, var_type))

        return col_value_type

    def as_relation_declaration(self) -> RelationDeclaration:
        return RelationDeclaration(self.relation_name, self.type_list)

    def has_same_terms_and_types(self, other: "Relation" # The other relation to compare with
                        ) -> bool: # True if same, false otherwise
        """
        Checks only term list and type list equivalence.
        """

        return self.type_list == other.type_list and self.term_list == other.term_list

    def get_index_of_free_var(self, free_var : DataTypes.free_var_name # the free var to search for
                            ) -> int: # the index of free_var in term_list.
        """
        @raise Exception: if free_var doesn't exist in term_list.
        """

        for i, term in enumerate(self.term_list):
            if term == free_var:
                return i
            
        raise Exception(f"{free_var} doesn't exist in term_list of relation {self.relation_name}"
                        f"term_list: {self.term_list}")

In [ ]:
show_doc(Relation.get_select_cols_values_and_types)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/ast_node_types.py#L97){target="_blank" style="float:right; font-size:smaller"}

### Relation.get_select_cols_values_and_types

>      Relation.get_select_cols_values_and_types ()

In [ ]:
show_doc(Relation.has_same_terms_and_types)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/ast_node_types.py#L109){target="_blank" style="float:right; font-size:smaller"}

### Relation.has_same_terms_and_types

>      Relation.has_same_terms_and_types (other:Relation)

Checks only term list and type list equivalence.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| other | 'Relation' | The other relation to compare with |
| **Returns** | **bool** | **True if same, false otherwise** |

In [ ]:
show_doc(Relation.get_index_of_free_var)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/ast_node_types.py#L117){target="_blank" style="float:right; font-size:smaller"}

### Relation.get_index_of_free_var

>      Relation.get_index_of_free_var (free_var:<DataTypes.free_var_name:3>)

@raise Exception: if free_var doesn't exist in term_list.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| free_var | DataTypes.free_var_name | the free var to search for |
| **Returns** | **int** | **the index of free_var in term_list.** |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
rel1 = Relation("parent", ["Michael", "Jack"], [DataTypes.string, DataTypes.string])
print(rel1)

parent("Michael", "Jack")


:::


In [ ]:
#| export
class IERelation:
    """
    a representation of an information extraction (ie) relation.
    An information extraction relation is different than a normal relation as it is constructed from
    the results of a call to an information extraction function.

    The ie relation instructs us on how to construct it:
    * its name is the ie function we need to call
    * its input term list represents a relation where each tuple is an argument list to call the ie function with.
    * its output term list represents a relation that filters the tuples that are returned from the ie function
    calls, and matches the values inside the tuples to free variables.
    """

    def __init__(self, relation_name: str, # the name of the information extraction relation
                       input_term_list: List[DataTypeMapping.term], #a list of the input terms for the ie functiom, must be either literal values or free variables
                       input_type_list: List[DataTypes], # a list of the term types in input_term_list
                       output_term_list: List[DataTypeMapping.term], # a list of the output terms for the ie function, must be either literal values or free variables
                       output_type_list: List[DataTypes] # a list of the term types in output_term_list
                       ): 
        """
        @raise Exception: if length of in/out term list doesn't match length of in/out type_list.
        """
        if len(input_term_list) != len(input_type_list):
            raise Exception(f"received different lengths of input_term_list ({len(input_term_list)}) "
                            f"and input_type_list ({len(input_type_list)})")
        if len(output_term_list) != len(output_type_list):
            raise Exception(f"received different lengths of output_term_list ({len(output_term_list)}) "
                            f"and output_type_list ({len(output_type_list)})")
        self.relation_name = relation_name
        self.input_term_list = input_term_list
        self.output_term_list = output_term_list
        self.input_type_list = input_type_list
        self.output_type_list = output_type_list

    def __str__(self) -> str:
        input_term_list_string = get_term_list_string(self.input_term_list, self.input_type_list)
        output_term_list_string = get_term_list_string(self.output_term_list, self.output_type_list)
        ie_relation_string = f"{self.relation_name}({input_term_list_string}) -> ({output_term_list_string})"
        return ie_relation_string

    def __repr__(self) -> str:
        return str(self)

    def get_term_list(self) -> List[DataTypeMapping.term]:
        return self.input_term_list + self.output_term_list

    def get_type_list(self) -> List[DataTypes]:
        return self.input_type_list + self.output_type_list

    def has_same_terms_and_types(self, other: Relation, # Other relation to compare with
                ) -> bool: # True if everything equivalent besides name, false otherwise
        """
        Checks that everything besides names is equivalent.
        """

        return self.output_type_list == other.type_list and self.output_term_list == other.term_list

In [ ]:
show_doc(IERelation.has_same_terms_and_types)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/ast_node_types.py#L180){target="_blank" style="float:right; font-size:smaller"}

### IERelation.has_same_terms_and_types

>      IERelation.has_same_terms_and_types (other:__main__.Relation)

Checks that everything besides names is equivalent.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| other | Relation | Other relation to compare with |
| **Returns** | **bool** | **True if everything equivalent besides name, false otherwise** |

::: {.callout-note collapse="true"}

##### Example

In [ ]:
ie_rel = IERelation("get_parents", ["X"], [DataTypes.free_var_name], ["Y"], [DataTypes.free_var_name])
print(ie_rel)

get_parents(X) -> (Y)


:::

In [ ]:
#| export
class AddFact(Relation):
    """
    a representation of an add_fact statement
    inherits from relation as a fact can be defined by a relation.
    """

    def __init__(self, relation_name: str, # the name of the relation
                        term_list: List[DataTypeMapping.term], # a list of the relation terms
                        type_list: Sequence[DataTypes]): # a list of the relation term types
        super().__init__(relation_name, term_list, type_list)

::: {.callout-note collapse="true"}

##### Example

In [ ]:
fact = AddFact("parent", ["Michael", "Jack"], [DataTypes.string, DataTypes.string])
print(fact)

parent("Michael", "Jack")


:::


In [ ]:
#| export
class RemoveFact(Relation):
    """
    a representation of a remove_fact statement
    inherits from relation as a fact can be defined by a relation.
    """

    def __init__(self, relation_name: str, # the name of the relation
                        term_list: List[DataTypeMapping.term], # a list of the relation terms
                        type_list: Sequence[DataTypes]): # a list of the relation term types
        super().__init__(relation_name, term_list, type_list)

::: {.callout-note collapse="true"}

##### Example

In [ ]:
fact = RemoveFact("parent", ["Michael", "Jack"], [DataTypes.string, DataTypes.string])
print(fact)

parent("Michael", "Jack")


:::

In [ ]:
#| export
class Query(Relation):
    """
    a representation of a query statement
    inherits from relation as a query can be defined by a relation
    """

    def __init__(self, relation_name: str, # the name of the relation
                        term_list: List[DataTypeMapping.term], # a list of the relation terms
                        type_list: Sequence[DataTypes]): # a list of the relation term types
        super().__init__(relation_name, term_list, type_list)

::: {.callout-note collapse="true"}

##### Example


In [ ]:
query = Query("parent", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name])
print(query)

parent(X, Y)


:::

In [ ]:
#| export
class Rule:
    """
    a representation of a rule statement.
    """

    def __init__(self,
                  head_relation: Relation, # the rule head, which is represented by a single relation
                    body_relation_list: List[Union[Relation, IERelation]], # a list of the rule body relations
                    body_relation_type_list: List[str] # a list of the rule body relations types (e.g. "relation", "ie_relation")
                    ):
        """
        @raise Exception: if length of term list doesn't match the length of type list.
        """
        if len(body_relation_list) != len(body_relation_type_list):
            raise Exception(f"received different lengths of body_relation_list ({len(body_relation_list)}) "
                            f"and body_relation_type_list ({len(body_relation_type_list)})")
        self.head_relation = head_relation
        self.body_relation_list = body_relation_list
        self.body_relation_type_list = body_relation_type_list
    
    def __str__(self) -> str:
        return f"{self.head_relation} <- {', '.join(map(str, self.body_relation_list))}"

    def __repr__(self) -> str:
        return str(self)

    def get_relations_by_type(self) -> Tuple[Set, Set]:
        """
        categorizes relations into two sets based on their types, distinguishing between regular relations and information-extraction relations.
        """
        relations, ie_relations = set(), set()
        for rel, rel_type in zip(self.body_relation_list, self.body_relation_type_list):
            if rel_type == "relation":
                relations.add(rel)
            else:
                ie_relations.add(rel)

        return relations, ie_relations

In [ ]:
show_doc(Rule.get_relations_by_type)

---

[source](https://github.com/DeanLight/spanner-workbench/blob/main/spannerlog/ast_node_types.py#L251){target="_blank" style="float:right; font-size:smaller"}

### Rule.get_relations_by_type

>      Rule.get_relations_by_type ()

categorizes relations into two sets based on their types, distinguishing between regular relations and information-extraction relations.

::: {.callout-note collapse="true"}

##### Example


In [ ]:
rule = Rule(
        "rule",
        [Relation("Person", ["X", "Y"], [DataTypes.free_var_name, DataTypes.free_var_name])],
        ["Relation"]
    )
print(rule)

rule <- Person(X, Y)


:::

In [ ]:
#| export
class Assignment:
    """
    a representation of an assignment statement.
    """

    def __init__(self, var_name: str, # the variable name to be assigned a value
                       value: DataTypeMapping.term, # the assigned value
                       value_type: DataTypes # the assigned value's type
            ):
        self.var_name = var_name
        self.value = value
        self.value_type = value_type
        
    def __str__(self) -> str:
        if self.value_type is DataTypes.string:
            # add quotes to a literal string value
            value_string = f'"{self.value}"'
        else:
            value_string = str(self.value)
        return f'{self.var_name} = {value_string}'

    def __repr__(self) -> str:
        return str(self)

::: {.callout-note collapse="true"}

##### Example

In [ ]:
assignment = Assignment("var1",10,int)
print(assignment)

var1 = 10


:::

In [ ]:
#| export
class ReadAssignment:
    """
    a representation of a read_assignment statement.
    """

    def __init__(self, var_name: str, # the variable name to be assigned a value
                       read_arg: str, # the argument that is passed to the read() function (e.g. "some_file" in `s = read("some_file")`)
                       read_arg_type: type # the type of the argument that is passed to the read function
            ):
        if read_arg_type not in [DataTypes.string, DataTypes.var_name]:
            raise TypeError(
                f'the argument that was passed to the read() function has an unexpected type: {read_arg_type}')

        self.var_name = var_name
        self.read_arg = read_arg
        self.read_arg_type = read_arg_type
    
    def __str__(self) -> str:
        if self.read_arg_type is DataTypes.string:
            # add quotes to a literal string argument
            read_arg_string = f'"{self.read_arg}"'
        else:
            read_arg_string = str(self.read_arg)
        return f'{self.var_name} = read({read_arg_string})'

    def __repr__(self) -> str:
        return str(self)

::: {.callout-note collapse="true"}

##### Example


In [ ]:
read_assignment = ReadAssignment("s", "file_name", DataTypes.string)
print(read_assignment)

s = read("file_name")


:::

#| hide
##### TEST

In [ ]:
#| hide
import pytest


In [ ]:
#| hide
# Test different lengths of term_list and type_list
term_list = ['X','Y',3]
type_list = [DataTypes.free_var_name,DataTypes.free_var_name]
with pytest.raises(Exception) as ex_info:
    term_list_str = get_term_list_string(term_list, type_list)

# Test declaring Relations
Relation("parents", ['X', 'Y'], [DataTypes.free_var_name, DataTypes.free_var_name])
Relation("parents", ['Jack', 'Michael'], [DataTypes.string, DataTypes.string])
Relation("parents", [2, 2], [DataTypes.integer, DataTypes.integer])
Relation("parents", ['[3,5)', 'Y'], [DataTypes.span, DataTypes.free_var_name])

# Test declaring illegal Relations
with pytest.raises(Exception):
    Relation("parents", ['X', 'Y'], [DataTypes.free_var_name])

with pytest.raises(Exception):
    Relation("parents", ['X'], [DataTypes.free_var_name, DataTypes.integer])
